In [1]:
%matplotlib notebook
from mpl_toolkits import mplot3d


import numpy as np
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data 
import time
import math
import random
import torch
import torch.nn as nn
from torch.autograd import Variable


In [2]:
physicsClient = p.connect(p.GUI)

In [3]:
p.setGravity(0,0,-10) 
p.resetSimulation() 
p.setAdditionalSearchPath(pybullet_data.getDataPath()) 
planeId = p.loadURDF("plane.urdf") 
robotId = p.loadURDF("iiwa7.urdf",flags=9, useFixedBase=1)

robotStartPos = [0,0,0]
robotStartOrientation = p.getQuaternionFromEuler([0,0,0])

p.resetBasePositionAndOrientation(robotId,robotStartPos,robotStartOrientation)

p.setJointMotorControlArray(robotId,range(7),p.VELOCITY_CONTROL,forces=np.zeros(7))

In [4]:
def simulate_system(x, u):
    x_next=[]
    for i in range(7):
        p.resetJointState(robotId,i,x[i],targetVelocity = x[i+7])
    
    p.setJointMotorControlArray(robotId,range(7), controlMode=p.TORQUE_CONTROL,forces=u)
    p.stepSimulation()
    for i in range(7):
        x_next.append(p.getJointStates(robotId,range(7))[i][0])
    for i in range(7):
        x_next.append(p.getJointStates(robotId,range(7))[i][1])
    x_next = np.array(x_next)
    return x_next

In [5]:
N=50
x = np.zeros([14,N])
u= np.zeros([7, N])
x_new=np.zeros([14,N])

In [6]:
for i in range(N):
    for j in range(7):
        a=random.randint(-180,180)#angule range(-pi,pi)
        b=random.uniform(-1, 10)#velocity range
        c=random.randint(-200,200)#torque range (-200,200)
        x[j,i]=math.radians(a/math.pi)
        x[j+7,i]=b
        u[j,i]=c
print(x)
print(u)

[[-5.66666667e-01  8.22222222e-01 -7.66666667e-01  4.77777778e-01
  -8.38888889e-01  6.27777778e-01 -5.38888889e-01 -7.61111111e-01
  -2.38888889e-01 -2.77777778e-01  4.94444444e-01  5.55555556e-02
  -8.94444444e-01  9.94444444e-01  0.00000000e+00 -7.05555556e-01
  -2.27777778e-01  7.22222222e-01 -1.83333333e-01  4.11111111e-01
   8.22222222e-01  5.55555556e-02  7.77777778e-01 -9.83333333e-01
   5.66666667e-01 -6.88888889e-01 -3.38888889e-01 -6.38888889e-01
  -5.77777778e-01  8.55555556e-01 -1.22222222e-01  9.72222222e-01
  -3.88888889e-01  4.44444444e-02  2.83333333e-01 -5.61111111e-01
  -6.83333333e-01 -5.22222222e-01  9.38888889e-01  9.44444444e-01
  -6.33333333e-01 -3.16666667e-01 -3.38888889e-01 -4.27777778e-01
   4.38888889e-01 -1.55555556e-01  2.22222222e-02 -9.11111111e-01
  -5.05555556e-01 -9.44444444e-02]
 [-4.55555556e-01 -1.83333333e-01  6.11111111e-02 -6.94444444e-01
  -3.33333333e-02 -7.66666667e-01  7.88888889e-01 -6.38888889e-01
  -8.61111111e-01 -5.05555556e-01  7.8333

In [7]:
for i in range(N):
    x_new[:,i]=simulate_system(x[:,i], u[:,i])
print("new state is:", x_new)

new state is: [[-5.41679816e-01  8.44511948e-01 -7.21969703e-01  4.76194602e-01
  -8.72844820e-01  6.42065414e-01 -5.11855637e-01 -7.48585696e-01
  -2.36487036e-01 -2.66603207e-01  5.20014710e-01  9.00130769e-02
  -9.56491779e-01  9.63619063e-01  3.75275623e-02 -6.97175714e-01
  -2.28802573e-01  7.67696745e-01 -1.90069981e-01  4.44704891e-01
   8.53945667e-01  1.20043177e-02  7.96367342e-01 -9.46509154e-01
   5.76704863e-01 -6.46467335e-01 -3.19921179e-01 -6.11515896e-01
  -5.79068545e-01  8.63074700e-01 -8.91874677e-02  9.70974934e-01
  -3.76061317e-01  4.34231996e-02  3.27468804e-01 -5.42948421e-01
  -6.42874308e-01 -4.96606277e-01  9.38942106e-01  9.70312294e-01
  -6.11937384e-01 -2.93127763e-01 -3.38779321e-01 -4.00985460e-01
   4.35353380e-01 -1.36281663e-01  2.11157644e-02 -8.69669780e-01
  -4.75418577e-01 -1.11674920e-01]
 [-4.41005201e-01 -1.55902375e-01  9.96824318e-02 -6.78278627e-01
  -7.02971692e-03 -7.26655303e-01  7.88133074e-01 -6.18072920e-01
  -8.57290135e-01 -4.979358

In [8]:
p.disconnect()

In [9]:
x=x.T
u = u.T
x_train = np.append(x,u,axis=1)
print(x_train)

[[-5.66666667e-01 -4.55555556e-01 -5.55555556e-01 ... -2.00000000e+00
   8.80000000e+01 -1.49000000e+02]
 [ 8.22222222e-01 -1.83333333e-01  1.66666667e-02 ...  3.10000000e+01
   1.88000000e+02  1.39000000e+02]
 [-7.66666667e-01  6.11111111e-02 -1.05555556e-01 ...  2.00000000e+00
  -6.30000000e+01  9.40000000e+01]
 ...
 [-9.11111111e-01  5.05555556e-01  5.00000000e-02 ... -1.09000000e+02
  -1.00000000e+00  9.00000000e+01]
 [-5.05555556e-01  5.44444444e-01  2.05555556e-01 ... -1.72000000e+02
   2.00000000e+02 -1.65000000e+02]
 [-9.44444444e-02  1.83333333e-01  5.55555556e-03 ...  1.12000000e+02
  -1.80000000e+02 -9.80000000e+01]]


In [10]:
y_train = x_new.T

In [11]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import nn,optim
from mpl_toolkits.mplot3d import Axes3D
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.transforms as transforms
torch.manual_seed(1)

In [12]:
#input_size = 1
input_size = 21
output_size = 14
num_epochs = 300
learning_rate = 0.002

x_train = x_train.astype(np.float32)
y_train=y_train.astype(np.float32)


x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)

In [13]:
class MultiLinearRegression(nn.Module):
    def __init__(self):
        super(MultiLinearRegression, self).__init__()
        self.linear = nn.Linear(21,14)  # 因为3个变量映射1个输出
        
    def forward(self,x):
        out = self.linear(x)
        return out

model = MultiLinearRegression()

In [14]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3)

epoch = 0
while True:
    output = model(x_train)  # 前向传播
    loss = criterion(output, y_train)  # 损失计算
    loss_value = loss.data.cpu().numpy()  # 获取损失值
    optimizer.zero_grad()  # 梯度置零
    loss.backward()  # 反向传播
    optimizer.step()  # 更新梯度
    
    epoch += 1
    if epoch % 100 == 0:  # 每100步打印一次损失
        print('Epoch:{}, loss:{:.6f}'.format(epoch, loss_value))
    if loss_value <= 1e-3:
        break
        


Epoch:100, loss:nan
Epoch:200, loss:nan
Epoch:300, loss:nan
Epoch:400, loss:nan
Epoch:500, loss:nan
Epoch:600, loss:nan
Epoch:700, loss:nan
Epoch:800, loss:nan
Epoch:900, loss:nan
Epoch:1000, loss:nan
Epoch:1100, loss:nan
Epoch:1200, loss:nan
Epoch:1300, loss:nan
Epoch:1400, loss:nan
Epoch:1500, loss:nan
Epoch:1600, loss:nan
Epoch:1700, loss:nan
Epoch:1800, loss:nan
Epoch:1900, loss:nan
Epoch:2000, loss:nan
Epoch:2100, loss:nan
Epoch:2200, loss:nan
Epoch:2300, loss:nan
Epoch:2400, loss:nan
Epoch:2500, loss:nan
Epoch:2600, loss:nan
Epoch:2700, loss:nan
Epoch:2800, loss:nan
Epoch:2900, loss:nan
Epoch:3000, loss:nan
Epoch:3100, loss:nan
Epoch:3200, loss:nan
Epoch:3300, loss:nan
Epoch:3400, loss:nan
Epoch:3500, loss:nan
Epoch:3600, loss:nan
Epoch:3700, loss:nan
Epoch:3800, loss:nan
Epoch:3900, loss:nan
Epoch:4000, loss:nan
Epoch:4100, loss:nan
Epoch:4200, loss:nan
Epoch:4300, loss:nan
Epoch:4400, loss:nan
Epoch:4500, loss:nan
Epoch:4600, loss:nan
Epoch:4700, loss:nan
Epoch:4800, loss:nan
E

Epoch:38100, loss:nan
Epoch:38200, loss:nan
Epoch:38300, loss:nan
Epoch:38400, loss:nan
Epoch:38500, loss:nan
Epoch:38600, loss:nan
Epoch:38700, loss:nan
Epoch:38800, loss:nan
Epoch:38900, loss:nan
Epoch:39000, loss:nan
Epoch:39100, loss:nan
Epoch:39200, loss:nan
Epoch:39300, loss:nan
Epoch:39400, loss:nan
Epoch:39500, loss:nan
Epoch:39600, loss:nan
Epoch:39700, loss:nan
Epoch:39800, loss:nan
Epoch:39900, loss:nan
Epoch:40000, loss:nan
Epoch:40100, loss:nan
Epoch:40200, loss:nan
Epoch:40300, loss:nan
Epoch:40400, loss:nan
Epoch:40500, loss:nan
Epoch:40600, loss:nan
Epoch:40700, loss:nan
Epoch:40800, loss:nan
Epoch:40900, loss:nan
Epoch:41000, loss:nan
Epoch:41100, loss:nan
Epoch:41200, loss:nan
Epoch:41300, loss:nan
Epoch:41400, loss:nan
Epoch:41500, loss:nan
Epoch:41600, loss:nan
Epoch:41700, loss:nan
Epoch:41800, loss:nan
Epoch:41900, loss:nan
Epoch:42000, loss:nan
Epoch:42100, loss:nan
Epoch:42200, loss:nan
Epoch:42300, loss:nan
Epoch:42400, loss:nan
Epoch:42500, loss:nan
Epoch:4260

Epoch:75500, loss:nan
Epoch:75600, loss:nan
Epoch:75700, loss:nan
Epoch:75800, loss:nan
Epoch:75900, loss:nan
Epoch:76000, loss:nan
Epoch:76100, loss:nan
Epoch:76200, loss:nan
Epoch:76300, loss:nan
Epoch:76400, loss:nan
Epoch:76500, loss:nan
Epoch:76600, loss:nan
Epoch:76700, loss:nan
Epoch:76800, loss:nan
Epoch:76900, loss:nan
Epoch:77000, loss:nan
Epoch:77100, loss:nan
Epoch:77200, loss:nan
Epoch:77300, loss:nan
Epoch:77400, loss:nan
Epoch:77500, loss:nan
Epoch:77600, loss:nan
Epoch:77700, loss:nan
Epoch:77800, loss:nan
Epoch:77900, loss:nan
Epoch:78000, loss:nan
Epoch:78100, loss:nan
Epoch:78200, loss:nan
Epoch:78300, loss:nan
Epoch:78400, loss:nan
Epoch:78500, loss:nan
Epoch:78600, loss:nan
Epoch:78700, loss:nan
Epoch:78800, loss:nan
Epoch:78900, loss:nan
Epoch:79000, loss:nan
Epoch:79100, loss:nan
Epoch:79200, loss:nan
Epoch:79300, loss:nan
Epoch:79400, loss:nan
Epoch:79500, loss:nan
Epoch:79600, loss:nan
Epoch:79700, loss:nan
Epoch:79800, loss:nan
Epoch:79900, loss:nan
Epoch:8000

Epoch:112200, loss:nan
Epoch:112300, loss:nan
Epoch:112400, loss:nan
Epoch:112500, loss:nan
Epoch:112600, loss:nan
Epoch:112700, loss:nan
Epoch:112800, loss:nan
Epoch:112900, loss:nan
Epoch:113000, loss:nan
Epoch:113100, loss:nan
Epoch:113200, loss:nan
Epoch:113300, loss:nan
Epoch:113400, loss:nan
Epoch:113500, loss:nan
Epoch:113600, loss:nan
Epoch:113700, loss:nan
Epoch:113800, loss:nan
Epoch:113900, loss:nan
Epoch:114000, loss:nan
Epoch:114100, loss:nan
Epoch:114200, loss:nan
Epoch:114300, loss:nan
Epoch:114400, loss:nan
Epoch:114500, loss:nan
Epoch:114600, loss:nan
Epoch:114700, loss:nan
Epoch:114800, loss:nan
Epoch:114900, loss:nan
Epoch:115000, loss:nan
Epoch:115100, loss:nan
Epoch:115200, loss:nan
Epoch:115300, loss:nan
Epoch:115400, loss:nan
Epoch:115500, loss:nan
Epoch:115600, loss:nan
Epoch:115700, loss:nan
Epoch:115800, loss:nan
Epoch:115900, loss:nan
Epoch:116000, loss:nan
Epoch:116100, loss:nan
Epoch:116200, loss:nan
Epoch:116300, loss:nan
Epoch:116400, loss:nan
Epoch:11650

Epoch:148000, loss:nan
Epoch:148100, loss:nan
Epoch:148200, loss:nan
Epoch:148300, loss:nan
Epoch:148400, loss:nan
Epoch:148500, loss:nan
Epoch:148600, loss:nan
Epoch:148700, loss:nan
Epoch:148800, loss:nan
Epoch:148900, loss:nan
Epoch:149000, loss:nan
Epoch:149100, loss:nan
Epoch:149200, loss:nan
Epoch:149300, loss:nan
Epoch:149400, loss:nan
Epoch:149500, loss:nan
Epoch:149600, loss:nan
Epoch:149700, loss:nan
Epoch:149800, loss:nan
Epoch:149900, loss:nan
Epoch:150000, loss:nan
Epoch:150100, loss:nan
Epoch:150200, loss:nan
Epoch:150300, loss:nan
Epoch:150400, loss:nan
Epoch:150500, loss:nan
Epoch:150600, loss:nan
Epoch:150700, loss:nan
Epoch:150800, loss:nan
Epoch:150900, loss:nan
Epoch:151000, loss:nan
Epoch:151100, loss:nan
Epoch:151200, loss:nan
Epoch:151300, loss:nan
Epoch:151400, loss:nan
Epoch:151500, loss:nan
Epoch:151600, loss:nan
Epoch:151700, loss:nan
Epoch:151800, loss:nan
Epoch:151900, loss:nan
Epoch:152000, loss:nan
Epoch:152100, loss:nan
Epoch:152200, loss:nan
Epoch:15230

...

Epoch:469900, loss:nan
Epoch:470000, loss:nan
Epoch:470100, loss:nan
Epoch:470200, loss:nan
Epoch:470300, loss:nan
Epoch:470400, loss:nan
Epoch:470500, loss:nan
Epoch:470600, loss:nan
Epoch:470700, loss:nan
Epoch:470800, loss:nan
Epoch:470900, loss:nan
Epoch:471000, loss:nan
Epoch:471100, loss:nan
Epoch:471200, loss:nan
Epoch:471300, loss:nan
Epoch:471400, loss:nan
Epoch:471500, loss:nan
Epoch:471600, loss:nan
Epoch:471700, loss:nan
Epoch:471800, loss:nan
Epoch:471900, loss:nan
Epoch:472000, loss:nan
Epoch:472100, loss:nan
Epoch:472200, loss:nan
Epoch:472300, loss:nan
Epoch:472400, loss:nan
Epoch:472500, loss:nan
Epoch:472600, loss:nan
Epoch:472700, loss:nan
Epoch:472800, loss:nan
Epoch:472900, loss:nan
Epoch:473000, loss:nan
Epoch:473100, loss:nan
Epoch:473200, loss:nan
Epoch:473300, loss:nan
Epoch:473400, loss:nan
Epoch:473500, loss:nan
Epoch:473600, loss:nan
Epoch:473700, loss:nan
Epoch:473800, loss:nan
Epoch:473900, loss:nan
Epoch:474000, loss:nan
Epoch:474100, loss:nan
Epoch:47420

Epoch:505900, loss:nan
Epoch:506000, loss:nan
Epoch:506100, loss:nan
Epoch:506200, loss:nan
Epoch:506300, loss:nan
Epoch:506400, loss:nan
Epoch:506500, loss:nan
Epoch:506600, loss:nan
Epoch:506700, loss:nan
Epoch:506800, loss:nan
Epoch:506900, loss:nan
Epoch:507000, loss:nan
Epoch:507100, loss:nan
Epoch:507200, loss:nan
Epoch:507300, loss:nan
Epoch:507400, loss:nan
Epoch:507500, loss:nan
Epoch:507600, loss:nan
Epoch:507700, loss:nan
Epoch:507800, loss:nan
Epoch:507900, loss:nan
Epoch:508000, loss:nan
Epoch:508100, loss:nan
Epoch:508200, loss:nan
Epoch:508300, loss:nan
Epoch:508400, loss:nan
Epoch:508500, loss:nan
Epoch:508600, loss:nan
Epoch:508700, loss:nan
Epoch:508800, loss:nan
Epoch:508900, loss:nan
Epoch:509000, loss:nan
Epoch:509100, loss:nan
Epoch:509200, loss:nan
Epoch:509300, loss:nan
Epoch:509400, loss:nan
Epoch:509500, loss:nan
Epoch:509600, loss:nan
Epoch:509700, loss:nan
Epoch:509800, loss:nan
Epoch:509900, loss:nan
Epoch:510000, loss:nan
Epoch:510100, loss:nan
Epoch:51020

Epoch:541800, loss:nan
Epoch:541900, loss:nan
Epoch:542000, loss:nan
Epoch:542100, loss:nan
Epoch:542200, loss:nan
Epoch:542300, loss:nan
Epoch:542400, loss:nan
Epoch:542500, loss:nan
Epoch:542600, loss:nan
Epoch:542700, loss:nan
Epoch:542800, loss:nan
Epoch:542900, loss:nan
Epoch:543000, loss:nan
Epoch:543100, loss:nan
Epoch:543200, loss:nan
Epoch:543300, loss:nan
Epoch:543400, loss:nan
Epoch:543500, loss:nan
Epoch:543600, loss:nan
Epoch:543700, loss:nan
Epoch:543800, loss:nan
Epoch:543900, loss:nan
Epoch:544000, loss:nan
Epoch:544100, loss:nan
Epoch:544200, loss:nan
Epoch:544300, loss:nan
Epoch:544400, loss:nan
Epoch:544500, loss:nan
Epoch:544600, loss:nan
Epoch:544700, loss:nan
Epoch:544800, loss:nan
Epoch:544900, loss:nan
Epoch:545000, loss:nan
Epoch:545100, loss:nan
Epoch:545200, loss:nan
Epoch:545300, loss:nan
Epoch:545400, loss:nan
Epoch:545500, loss:nan
Epoch:545600, loss:nan
Epoch:545700, loss:nan
Epoch:545800, loss:nan
Epoch:545900, loss:nan
Epoch:546000, loss:nan
Epoch:54610

Epoch:577600, loss:nan
Epoch:577700, loss:nan
Epoch:577800, loss:nan
Epoch:577900, loss:nan
Epoch:578000, loss:nan
Epoch:578100, loss:nan
Epoch:578200, loss:nan
Epoch:578300, loss:nan
Epoch:578400, loss:nan
Epoch:578500, loss:nan
Epoch:578600, loss:nan
Epoch:578700, loss:nan
Epoch:578800, loss:nan
Epoch:578900, loss:nan
Epoch:579000, loss:nan
Epoch:579100, loss:nan
Epoch:579200, loss:nan
Epoch:579300, loss:nan
Epoch:579400, loss:nan
Epoch:579500, loss:nan
Epoch:579600, loss:nan
Epoch:579700, loss:nan
Epoch:579800, loss:nan
Epoch:579900, loss:nan
Epoch:580000, loss:nan
Epoch:580100, loss:nan
Epoch:580200, loss:nan
Epoch:580300, loss:nan
Epoch:580400, loss:nan
Epoch:580500, loss:nan
Epoch:580600, loss:nan
Epoch:580700, loss:nan
Epoch:580800, loss:nan
Epoch:580900, loss:nan
Epoch:581000, loss:nan
Epoch:581100, loss:nan
Epoch:581200, loss:nan
Epoch:581300, loss:nan
Epoch:581400, loss:nan
Epoch:581500, loss:nan
Epoch:581600, loss:nan
Epoch:581700, loss:nan
Epoch:581800, loss:nan
Epoch:58190

KeyboardInterrupt: 

In [ ]:
w = model.linear.weight.data.numpy()
b = model.linear.bias.data.numpy()
print('w:{},b:{}'.format(w,b))
print(loss)